In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_ids = test["PassengerId"]

def clean(data):
    data = data.drop(["Name", "Ticket", "Cabin", "PassengerId"], axis=1)
    cols = ["SibSp", "Parch", "Age", "Fare"]
    
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
        
    data.Embarked.fillna("U", inplace=True)
    return data

data = clean(data)
test = clean(test)

In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)
    


['female' 'male']
['C' 'Q' 'S' 'U']


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = data["Survived"]
x = data.drop("Survived", axis=1)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(x_train, y_train)

In [6]:
predictions = clf.predict(x_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8100558659217877

In [7]:
submission_preds = clf.predict(test)

In [8]:
df = pd.DataFrame({"PassengerId":test_ids.values,
                  "Survived": submission_preds})

In [9]:
df.to_csv("Submission.csv", index=False)